In [1]:
# Import dependencies
import pandas as pd
import numpy as np

In [2]:
# Read schools file
df_schools = pd.read_csv('./datasource/schools_complete.csv')
df_schools.head() 

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
# Summarize Schools data.
dic = {'School ID': 'nunique', 'size':'sum' , 'budget':'sum'}
sum_schools = pd.DataFrame(df_schools.agg(dic)).T

# Rename columns
rename_cols = {"School ID":"Total Schools", "size":"Total Students", "budget":"Total Budget"}
sum_schools.rename(columns=rename_cols, inplace=True)
sum_schools

,Total Schools,Total Students,Total Budget
0,15,39170,24649428


In [4]:
# Head students file
df_students = pd.read_csv('./datasource/students_complete.csv')

# Add columns for pass math and reading
df_students['pass_math'] = df_students['math_score'].apply(lambda score: 1 if score >=70 else 0)
df_students['pass_read'] = df_students['reading_score'].apply(lambda score: 1 if score >=70 else 0)

df_students.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,pass_math,pass_read
0,0,Paul Bradley,M,9th,Huang High School,66,79,1,0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,1
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,1
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,1,1


In [5]:
# Summarize Students data.
dic = {"Student ID": "nunique", "math_score":"mean",  'reading_score':'mean',  'pass_math':'sum',  'pass_read':'sum'}
sum_students = pd.DataFrame(df_students.agg(dic)).T
# Rename columns
rename_cols = {"Student ID":"Total Students", "math_score":"Average Math Score", "reading_score":"Average Reading Score"}
sum_students.rename(columns=rename_cols, inplace=True) 
sum_students

,Total Students,Average Math Score,Average Reading Score,pass_math,pass_read
0,39170.0,78.985371,81.87784,29370.0,33610.0


In [6]:
# Compute Students Percentages
sum_students["% Passing Math"] = sum_students["pass_math"] / sum_students['Total Students'] * 100
sum_students["% Passing Reading"] =  sum_students["pass_read"] / sum_students['Total Students'] * 100
# Really??? Should be "% Passing Math" + "% Passing Reading", is not it?
sum_students["% Overall Passing Rate"] = (sum_students["Average Math Score"] + sum_students["Average Reading Score"]) / 2
sum_students

,Total Students,Average Math Score,Average Reading Score,pass_math,pass_read,% Passing Math,% Passing Reading,% Overall Passing Rate
0,39170.0,78.985371,81.87784,29370.0,33610.0,74.980853,85.805463,80.431606


In [7]:
# Merge Schools and Students summaries 
total_summary = pd.merge(sum_schools , sum_students, on="Total Students", how="outer")

# Format before print
total_summary["Total Budget"] = total_summary["Total Budget"].astype(float).map("${:,.2f}".format)
total_summary["Total Students"] = total_summary["Total Students"].astype(int).map("{:,.0f}".format)
total_summary["% Passing Math"] = total_summary["% Passing Math"].astype(float).map("{:.2f}%".format)
total_summary["% Passing Reading"] = total_summary["% Passing Reading"].astype(float).map("{:.2f}%".format)
total_summary["% Overall Passing Rate"] = total_summary["% Overall Passing Rate"].astype(float).map("{:.2f}%".format)
# Drop unnecessary columns
total_summary.drop(['pass_math','pass_read'], axis=1, inplace=True)

# District Summary

In [8]:
total_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,80.43%


# Starting School Summary

In [9]:
# Merging main tables
main_tbl = pd.merge(df_schools, df_students, how="left", on="school_name")
main_tbl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39170 entries, 0 to 39169
Data columns (total 13 columns):
School ID        39170 non-null int64
school_name      39170 non-null object
type             39170 non-null object
size             39170 non-null int64
budget           39170 non-null int64
Student ID       39170 non-null int64
student_name     39170 non-null object
gender           39170 non-null object
grade            39170 non-null object
reading_score    39170 non-null int64
math_score       39170 non-null int64
pass_math        39170 non-null int64
pass_read        39170 non-null int64
dtypes: int64(8), object(5)
memory usage: 4.2+ MB


In [10]:
# Add columns for pass math and reading
main_tbl.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score,pass_math,pass_read
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,1,0
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,0,1
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,0,1
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,0,0
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,1,1


In [11]:
# Create an overview table that summarizes key metrics about each school
aggregation = {"Student ID": "count", "math_score":"mean",  'reading_score':'mean',"pass_math":"sum","pass_read":"sum"}
renamecol =   {"Student ID": "Total Students", "math_score":"Average Math Score",  'reading_score':'Average Reading Score', \
               "school_name":"School Name","type":"School Type","budget":"Total School Budget"}
agg_by_school = main_tbl.groupby(["School ID","school_name","type","budget"],axis=0).agg(aggregation)
agg_by_school.reset_index(inplace=True)
agg_by_school.set_index("School ID",inplace=True)
agg_by_school.rename(columns=renamecol,inplace=True)
agg_by_school["Per Student Budget"] = agg_by_school['Total School Budget']/agg_by_school['Total Students']
agg_by_school["% Passing Math"] = agg_by_school['pass_math']/agg_by_school['Total Students'] * 100
agg_by_school["% Passing Reading"] = agg_by_school['pass_read']/agg_by_school['Total Students'] * 100
agg_by_school["% Overall Passing Rate"] = (agg_by_school['% Passing Math'] + agg_by_school['% Passing Reading']) / 2 

# Format output.
agg_by_school["Total Students"] = agg_by_school["Total Students"].astype(float).map("{:,.0f}".format)
agg_by_school["Total School Budget"] = agg_by_school["Total School Budget"].astype(float).map("${:,.2f}".format)
agg_by_school["Per Student Budget"] = agg_by_school["Per Student Budget"].astype(float).map("${:,.2f}".format)
agg_by_school["% Passing Math"] = agg_by_school["% Passing Math"].astype(float).map("{:.2f}%".format)
agg_by_school["% Passing Reading"] = agg_by_school["% Passing Reading"].astype(float).map("{:.2f}%".format)
agg_by_school["% Overall Passing Rate"] = agg_by_school["% Overall Passing Rate"].astype(float).map("{:.3f}%".format)

# Remove index from "School ID", it will be dropped.
agg_by_school.reset_index(inplace=True)
# Drop unnecessary columns.
agg_by_school.drop(['pass_math','pass_read', 'School ID'], axis = 1, inplace = True)
# Set the index to "School Name".
agg_by_school.set_index("School Name",inplace=True)
# agg_by_school.head()


# Top Performing Schools (By Passing Rate)

In [12]:
agg_by_school.sort_values("% Overall Passing Rate",ascending=False).head(5)

,School Type,Total School Budget,Total Students,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356.00","1,858",83.061895,83.975780,$582.00,94.13%,97.04%,95.587%
Thomas High School,Charter,"$1,043,130.00","1,635",83.418349,83.848930,$638.00,93.27%,97.31%,95.291%
Pena High School,Charter,"$585,858.00",962,83.839917,84.044699,$609.00,94.59%,95.95%,95.270%
Griffin High School,Charter,"$917,500.00","1,468",83.351499,83.816757,$625.00,93.39%,97.14%,95.266%
Wilson High School,Charter,"$1,319,574.00","2,283",83.274201,83.989488,$578.00,93.87%,96.54%,95.204%


# Bottom Performing Schools (By Passing Rate)

In [13]:
agg_by_school.sort_values("% Overall Passing Rate",ascending=True).head(5)

,School Type,Total School Budget,Total Students,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"$2,547,363.00","3,999",76.842711,80.744686,$637.00,66.37%,80.22%,73.293%
Figueroa High School,District,"$1,884,411.00","2,949",76.711767,81.158020,$639.00,65.99%,80.74%,73.364%
Huang High School,District,"$1,910,635.00","2,917",76.629414,81.182722,$655.00,65.68%,81.32%,73.500%
Johnson High School,District,"$3,094,650.00","4,761",77.072464,80.966394,$650.00,66.06%,81.22%,73.640%
Ford High School,District,"$1,763,916.00","2,739",77.102592,80.746258,$644.00,68.31%,79.30%,73.804%


# Math Scores by Grade

In [14]:
# Create DataFrame for Math Score average Grouping By School ID and Grade  
grade_math_score = main_tbl.pivot_table(index=["school_name"], columns='grade', values='math_score', aggfunc='mean') 

# Sort Grade and rename titles name for presentation.
columns = ['9th', '10th','11th','12th']
grade_math_score = grade_math_score[columns]
grade_math_score.columns.name = 'Grade'
grade_math_score.index.name = 'School Name'
grade_math_score

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Reading Scores by Grade

In [15]:
# Create DataFrame for Reading Score average Grouping By School ID and Grade  
grade_reading_score = main_tbl.pivot_table(index=["school_name"], columns='grade', values='reading_score', aggfunc='mean') 

# Sort Grade and rename titles name for presentation.
columns = ['9th', '10th','11th','12th']
grade_reading_score = grade_reading_score[columns]
grade_reading_score.columns.name = 'Grade'
grade_reading_score.index.name = 'School Name'

grade_reading_score

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [16]:
main_tbl.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score,pass_math,pass_read
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,1,0
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,0,1
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,0,1
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,0,0
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,1,1
